In [ ]:
import open3d as o3d
import numpy as np
from matplotlib import pyplot as plt
import os

In [ ]:
ZED_data = '../../data/20221210/ZED/720/point_cloud_PLY_3029_720_06-12-2022-19-39-57.ply'
os.path.exists(ZED_data)

In [ ]:
def appendSpherical_np(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    xy = xyz[:,2]**2 + xyz[:,1]**2
    ptsnew[:,3] = np.sqrt(xy + xyz[:,0]**2)
    # ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,0]) # for elevation angle defined from Z-axis down
    ptsnew[:,4] = np.arctan2(xyz[:,0], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,2])
    return ptsnew

In [ ]:
pcd_zed = o3d.io.read_point_cloud(ZED_data)
pts_zed = np.asarray(pcd_zed.points)

In [ ]:
pts_zed_sph = appendSpherical_np(pts_zed)[:,3:6]
pts_zed_sph[:,1:3] = np.degrees(pts_zed_sph[:,1:3]) + 180

In [ ]:
# pts_zed_sph[:, 1] = np.radians(pts_zed_sph[:,1])
# pts_zed_sph[:, 2] = np.radians(pts_zed_sph[:, 2])

In [ ]:
# # Convert spherical coordinates to Cartesian coordinates
# x = pts_zed_sph[:, 0] * np.cos(pts_zed_sph[:, 1]) * np.cos(pts_zed_sph[:, 2])
# y = pts_zed_sph[:, 0] * np.cos(pts_zed_sph[:, 1]) * np.sin(pts_zed_sph[:, 2])
# z = pts_zed_sph[:, 0] * np.sin(pts_zed_sph[:, 1])

# zed_pts = np.asarray([x, y, z]).T

# pcd__back_zed = o3d.geometry.PointCloud()
# pcd__back_zed.points = o3d.utility.Vector3dVector(zed_pts)

In [ ]:
# o3d.visualization.draw_geometries([
#     pcd__back_zed, 
#     ])

In [ ]:
a = pts_zed_sph[:,2]

lower_angle = 20
upper_angle = 20

polar_min, polar_max = pts_zed_sph[:,2].min() + lower_angle, pts_zed_sph[:,2].max() - upper_angle

pts_zed_sph_filt = pts_zed_sph[(a < polar_min) | (a > polar_max)]
# pts_zed_sph_filt = b[b[:,2] < max]

print(pts_zed_sph_filt.shape)

In [ ]:
# pts_zed_sph_filt[:, 1] = np.radians(pts_zed_sph_filt[:,1])
# pts_zed_sph_filt[:, 2] = np.radians(pts_zed_sph_filt[:, 2])

In [ ]:
# # Convert spherical coordinates to Cartesian coordinates
# x = pts_zed_sph_filt[:, 0] * np.cos(pts_zed_sph_filt[:, 1]) * np.cos(pts_zed_sph_filt[:, 2])
# y = pts_zed_sph_filt[:, 0] * np.cos(pts_zed_sph_filt[:, 1]) * np.sin(pts_zed_sph_filt[:, 2])
# z = pts_zed_sph_filt[:, 0] * np.sin(pts_zed_sph_filt[:, 1])

# zed_pts = np.asarray([x, y, z]).T

# pcd__back_zed = o3d.geometry.PointCloud()
# pcd__back_zed.points = o3d.utility.Vector3dVector(zed_pts)

In [ ]:
# o3d.visualization.draw_geometries([
#     pcd__back_zed, 
#     ])

In [ ]:
def find_closest_differences(arr):
    sorted_arr = sorted(arr.astype(float))
    diffs = np.asanyarray([sorted_arr[i+1] - sorted_arr[i] for i in range(len(sorted_arr)-1)])
    return min(diffs[diffs > 1e-2]), max(diffs[diffs > 1e-2])

In [ ]:
pts_zed_sph_filt[:,1] *= 14.03
pts_zed_sph_filt[:,1] = pts_zed_sph_filt[:,1]//1 - 1903
print(len(np.unique(pts_zed_sph_filt[:,1])))
print(find_closest_differences(pts_zed_sph_filt[:,1]))

h_range = (int(pts_zed_sph_filt[:,1].min()), int(pts_zed_sph_filt[:,1].max()+1))

In [ ]:
bg_depth = pts_zed_sph_filt[:,0].mean()
print(bg_depth)

In [ ]:
def map_range_to_interval(value, old_min, old_max, new_min, new_max):
    mapped_value = (value - old_min) * (new_max - new_min) / (old_max - old_min) + new_min
    return int(mapped_value)

In [ ]:
mask = pts_zed_sph_filt[:,2] > lower_angle

pts_zed_sph_filt[mask, 2] -= 360
pts_zed_sph_filt[not mask.all(), 2] += lower_angle

pts_zed_sph_filt[:,2] *= 1.6

v_range = (pts_zed_sph_filt[:,2].min(), pts_zed_sph_filt[:,2].max()+1)

pts_zed_sph_filt[:,2] = np.vectorize(map_range_to_interval)(pts_zed_sph_filt[:,2], v_range[0], v_range[1], 0, 720)
# np.savetxt("zed_filt.csv", pts_zed_sph_filt,delimiter=',')

In [ ]:
print(len(np.unique(pts_zed_sph_filt[:,2])))
print(find_closest_differences(pts_zed_sph_filt[:,2]))

In [ ]:
sph_zed_frame = []
for i in range(0, 720):
    lower = i
    upper = i + 1
    mask = (pts_zed_sph_filt[:, 2] >= lower) & (pts_zed_sph_filt[:, 2] < upper)
    sub_array = pts_zed_sph_filt[mask, 0:2]

    row = np.ones(1280)*bg_depth
    row[sub_array[:,1].astype(int)] = sub_array[:,0]
    sph_zed_frame.append(row)

In [ ]:
np.savetxt("sph_zed_frame.csv", sph_zed_frame,delimiter=',')

In [ ]:
plt.imshow(sph_zed_frame)

In [ ]:
# plt.imsave("../../data/ZED/720/zed_sph_frame.png", sph_zed_frame)

In [ ]:
sph_zed_frame = np.asarray(sph_zed_frame)
print(sph_zed_frame.shape)

In [ ]:
def back_to_pts_form(arr):
    # get the shape of the input array
    m, n = arr.shape
    azimuth_const = 90/n
    polar_const = 30/m
    
    # create a 3D output array of size (m * n, 3)
    out = np.zeros((m * n, 3))
    
    # populate the output array
    for row in range(m):
        for col in range(n):
            index = row * n + col
            out[index, 0] = arr[row, col]
            out[index, 1] = row * polar_const
            out[index, 2] = col * azimuth_const 
    
    return out

In [ ]:
back_pts_zed = back_to_pts_form(sph_zed_frame)

back_pts_zed[:,1] = np.radians(back_pts_zed[:,1])
back_pts_zed[:,2] = np.radians(back_pts_zed[:,2])

In [ ]:
point_cloud_data = back_pts_zed

# Convert spherical coordinates to Cartesian coordinates
x = point_cloud_data[:, 0] * np.cos(point_cloud_data[:, 1]) * np.cos(point_cloud_data[:, 2])
y = point_cloud_data[:, 0] * np.cos(point_cloud_data[:, 1]) * np.sin(point_cloud_data[:, 2])
z = point_cloud_data[:, 0] * np.sin(point_cloud_data[:, 1])

pg_pts_zed = np.asarray([x, y, z]).T

pcd_pg_zed = o3d.geometry.PointCloud()
pcd_pg_zed.points = o3d.utility.Vector3dVector(pg_pts_zed)

In [ ]:
o3d.visualization.draw_geometries([pcd_pg_zed])